# Notes:

In building this model, I have separated out running code for **new** phages into a separate package called `bacphlip`. To ensure that this code-base gives the same results as when it's all thrown together here, I'm running a brief triple check to ensure that if I treat my "test set" of phages as if they were new, the results are equivalent. 

It's probably a good idea to run this with any updates.

# Imports

In [1]:
import pandas as pd
import os
import shutil
import glob
import numpy as np

import subprocess

In [2]:
temp_dir = '/Users/adamhockenberry/Projects/bacphlip-model-dev/Data/bacphlip_test/'
if not os.path.exists(temp_dir):
    os.mkdir(temp_dir)

In [3]:
orig_fasta_path = '../Data/phage_data_nmicro2017/phage_fasta_files/'
test_df = pd.read_csv('../Data/classifier_data/test_df.csv', index_col=0)
for i in test_df['Identifier_AJH']:
    shutil.copyfile(orig_fasta_path+i+'.fasta', temp_dir+i+'.fasta')

In [4]:
# for i in test_df['Identifier_AJH'][:10]:
#     fasta_file = temp_dir+i+'.fasta'
#     print(fasta_file)
#     subprocess.run(args=['python',\
#                          '/Users/adamhockenberry/Projects/bacphlip-py/bacphlip/bacphlip.py',\
#                          '-i',\
#                          '{}'.format(fasta_file),\
#                          '-f'],\
#                   check=True)

In [5]:
import sys
sys.path.append('../../bacphlip-py/')
import bacphlip
for i in test_df['Identifier_AJH'][:10]:
    fasta_file = temp_dir+i+'.fasta'
    print(fasta_file)
    bacphlip.run_pipeline('{}'.format(fasta_file), force_overwrite=True)

/Users/adamhockenberry/Projects/bacphlip-model-dev/Data/bacphlip_test/eagleeye.fasta
#################################################################################
Beginning BACPHLIP pipeline
Finished six frame translation of genome (nucleotide) with output stored in /Users/adamhockenberry/Projects/bacphlip-model-dev/Data/bacphlip_test/eagleeye.fasta.6frame
Finished outside call to hmmsearch with output stored in /Users/adamhockenberry/Projects/bacphlip-model-dev/Data/bacphlip_test/eagleeye.fasta.hmmsearch
Finished converting hmmsearch with output stored in /Users/adamhockenberry/Projects/bacphlip-model-dev/Data/bacphlip_test/eagleeye.fasta.hmmsearch.tsv
Finished with BACPHLIP predictions! Final output file stored in /Users/adamhockenberry/Projects/bacphlip-model-dev/Data/bacphlip_test/eagleeye.fasta.bacphlip
#################################################################################
/Users/adamhockenberry/Projects/bacphlip-model-dev/Data/bacphlip_test/seagreen.fasta
#########

Finished outside call to hmmsearch with output stored in /Users/adamhockenberry/Projects/bacphlip-model-dev/Data/bacphlip_test/michellemybell.fasta.hmmsearch
Finished converting hmmsearch with output stored in /Users/adamhockenberry/Projects/bacphlip-model-dev/Data/bacphlip_test/michellemybell.fasta.hmmsearch.tsv
Finished with BACPHLIP predictions! Final output file stored in /Users/adamhockenberry/Projects/bacphlip-model-dev/Data/bacphlip_test/michellemybell.fasta.bacphlip
#################################################################################


# Compare results!

In [6]:
###Load classifier model
import joblib
clf = joblib.load('../Data/classifier_data/rf_highMinAJH.joblib')
test_df = pd.read_csv('../Data/classifier_data/test_df.csv', index_col=0)
test_res = clf.predict_proba(test_df[test_df.columns[23:]])
a = list(zip(*test_res))[0]
b = list(zip(*test_res))[1]
test_df['Virulent'] = a
test_df['Temperate'] = b

In [7]:
bacphlip_dir = '../Data/bacphlip_test/'
for index in test_df.index[:20]:
    ident = test_df.loc[index]['Identifier_AJH']
    bacphlip_out = bacphlip_dir + ident + '.fasta.bacphlip'
    res_df = pd.read_csv(bacphlip_out, sep='\t')
    vals = res_df.loc[0].values
    assert np.isclose(vals[0], test_df.loc[index]['Virulent'])
    assert np.isclose(vals[1], test_df.loc[index]['Temperate'])
    print('success')
    

success
success
success
success
success
success
success
success
success
success


FileNotFoundError: [Errno 2] File b'../Data/bacphlip_test/arturo.fasta.bacphlip' does not exist: b'../Data/bacphlip_test/arturo.fasta.bacphlip'

In [ ]:
vals

In [ ]:
test_df.loc[index]['Virulent']

In [ ]:
test_df.loc[index]['Temperate']